In [ ]:
%%bash

docker run -p 127.0.0.1:39200:9200 -d --name elasticsearch \
  -e ELASTIC_PASSWORD=password \
  -e "discovery.type=single-node" \
  -e "xpack.security.http.ssl.enabled=false" \
  -e "xpack.license.self_generated.type=trial" \
  docker.elastic.co/elasticsearch/elasticsearch:8.15.2

In [ ]:
%pip install langchain-nomic langchain_aws langchain_community tiktoken langchainhub langchain-elasticsearch langchain langgraph gpt4all langchain-text-splitters boto3

# Index the summaries in ElasticSearch Vector Store



In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_elasticsearch import ElasticsearchStore
from langchain_text_splitters import RecursiveCharacterTextSplitter

files = [
    "alpha_project_meeting_action_items.txt",
    "alpha_project_meeting_key_points.txt"
]

docs = [TextLoader(file).load() for file in files]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)
documents=doc_splits

In [7]:
import boto3
import os
from langchain_aws.embeddings.bedrock import BedrockEmbeddings

bedrock = boto3.client('bedrock-runtime', region_name=os.getenv('AWS_REGION', 'us-east-1'))

embeddings=BedrockEmbeddings(
        model_id = "amazon.titan-embed-text-v2:0",
        client=bedrock)


In [8]:

db = ElasticsearchStore.from_documents(
    documents,
    embeddings,
    es_url="http://127.0.0.1:39200",
    es_user="elastic",
    es_password="password",
    index_name="summarize-me",
)
retriever = db.as_retriever()

In [9]:
from langchain_aws import ChatBedrock
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# LLM
llm = ChatBedrock(model_id = "anthropic.claude-3-haiku-20240307-v1:0",                   
                  client=bedrock,
                  beta_use_converse_api=True)

prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance 
    of a retrieved document to a user question. If the document contains keywords related to the user question, 
    grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question", "document"],
)

question = "What are the key points and action items?"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
doc_txt

'- Coordinate with the back end team to resolve the API integration issue by Wednesday.\n- Finalize the marketing content and share it with the team by Friday for feedback.\n- Schedule and conduct the sales team training sessions over the next two weeks.\n- Follow up on resource allocation and keep everyone updated on any changes.\n- Schedule a cross departmental meeting next week to ensure alignment as the launch date approaches.'

# Retrieval Grader (Optional)
Check if the results are relevant to the question.

In [10]:
retrieval_grader = prompt | llm | JsonOutputParser()
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

{'score': 'yes'}


# Generate


In [11]:

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# Prompt
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
    Keep the answer concise <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question} 
    Context: {context} 
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
question = "What are the key points and action items?"

docs = retriever.invoke(question)
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

Based on the provided context, the key points and action items are:

Key Points:
- Status of the alpha project
- Marketing strategy for the product launch
- Sales projections based on current market trends
- Customer feedback from the beta testers
- Upcoming features in the marketing materials

Action Items:
- Coordinate with the back end team to resolve the API integration issue by Wednesday.
- Finalize the marketing content and share it with the team by Friday for feedback.
- Schedule and conduct the sales team training sessions over the next two weeks.
- Follow up on resource allocation and keep everyone updated on any changes.
- Schedule a cross departmental meeting next week to ensure alignment as the launch date approaches.

<|eot_id|><|start_header_id|>user<|end_header_id|>


# Grounded Grader (Optional)
Evaluate if the generation is grounded in facts (e.g. not hallucinating).


In [12]:
prompt = PromptTemplate(
    template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether 
    an answer is grounded in / supported by a set of facts. Give a binary 'yes' or 'no' score to indicate 
    whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a 
    single key 'score' and no preamble or explanation. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here are the facts:
    \n ------- \n
    {documents} 
    \n ------- \n
    Here is the answer: {generation}  <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "documents"],
)

hallucination_grader = prompt | llm | JsonOutputParser()
hallucination_grader.invoke({"documents": docs, "generation": generation})


{'score': 'yes'}

# Usefulness Grader (Optional)
Evaluate if the generation is useful.

In [14]:
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether an 
    answer is useful to resolve a question. Give a binary score 'yes' or 'no' to indicate whether the answer is 
    useful to resolve a question. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|> Here is the answer:
    \n ------- \n
    {generation} 
    \n ------- \n
    Here is the question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "question"],
)

answer_grader = prompt | llm | JsonOutputParser()
answer_grader.invoke({"question": question, "generation": generation})


{'score': 'yes'}